In [8]:
import os
import tarfile
import urllib.request
from pathlib import Path

DOWNLOAD_ROOT = "http://spamassassin.apache.org/old/publiccorpus/"
HAM_URL = DOWNLOAD_ROOT + "20030228_easy_ham.tar.bz2"
SPAM_URL = DOWNLOAD_ROOT + "20030228_spam.tar.bz2"
DATA_PATH = Path('datasets')
SPAM_PATH = DATA_PATH / 'spam'

In [2]:
if not DATA_PATH.is_dir():
    DATA_PATH.mkdir()
if not SPAM_PATH.is_dir():
    SPAM_PATH.mkdir()

In [20]:
def fetch_spam_data(spam_url=SPAM_URL, spam_path=SPAM_PATH):
    for filename,url in [('ham.tar.bz2',HAM_URL),('spam.tar.bz2',SPAM_URL)]:
        path=SPAM_PATH / filename
        if not path.is_file():
            with urllib.request.urlopen(url) as response:
                downloadfile = response.read()
                with open(path,'wb') as f:
                    f.write(downloadfile)
        with tarfile.open(path) as tf:
            tf.extractall(path=SPAM_PATH)

In [21]:
fetch_spam_data()

In [23]:
os.listdir(SPAM_PATH)

['easy_ham', 'ham.tar.bz2', 'spam', 'spam.tar.bz2']

In [24]:
HAM_DIR = SPAM_PATH / 'easy_ham'
SPAM_DIR = SPAM_PATH / 'spam'
ham_filenames = [name for name in os.listdir(HAM_DIR) if len(name)>10]
spam_filename = [name for name in os.listdir(SPAM_DIR) if len(name)>10]

In [25]:
len(ham_filenames), len(spam_filename)

(2500, 500)

In [26]:
import email
import email.policy

In [27]:
def load_email(is_spam, filename, spam_path=SPAM_PATH):
    directory = 'spam' if is_spam else 'easy_ham'
    with open(spam_path/directory/filename, 'rb') as f:
        return email.parser.BytesParser(policy=email.policy.default).parse(f)

In [28]:
ham_emails = [load_email(False,name) for name in ham_filenames]
spam_emails = [load_email(True,name) for name in spam_filename]

In [31]:
print(ham_emails[0].get_content().strip())

Date:        Wed, 21 Aug 2002 10:54:46 -0500
    From:        Chris Garrigues <cwg-dated-1030377287.06fa6d@DeepEddy.Com>
    Message-ID:  <1029945287.4797.TMDA@deepeddy.vircio.com>


  | I can't reproduce this error.

For me it is very repeatable... (like every time, without fail).

This is the debug log of the pick happening ...

18:19:03 Pick_It {exec pick +inbox -list -lbrace -lbrace -subject ftp -rbrace -rbrace} {4852-4852 -sequence mercury}
18:19:03 exec pick +inbox -list -lbrace -lbrace -subject ftp -rbrace -rbrace 4852-4852 -sequence mercury
18:19:04 Ftoc_PickMsgs {{1 hit}}
18:19:04 Marking 1 hits
18:19:04 tkerror: syntax error in expression "int ...

Note, if I run the pick command by hand ...

delta$ pick +inbox -list -lbrace -lbrace -subject ftp -rbrace -rbrace  4852-4852 -sequence mercury
1 hit

That's where the "1 hit" comes from (obviously).  The version of nmh I'm
using is ...

delta$ pick -version
pick -- nmh-1.0.4 [compiled on fuchsia.cs.mu.OZ.AU at Sun Mar 17 14:55:56 

In [33]:
(ham_emails[0].get_content_type())

'text/plain'

In [34]:
email = ham_emails[0]

In [ ]:
email.get_payload()